# Intro

There are two separate definitions here:
- [file upload](https://groups.google.com/d/msg/jupyter/EmsrRq3bs3M/Fo6_BSCqBwAJ)
- file selection
    
The first one is for the case of a deployed app running on some remote server, and the user wants to upload some files there. Potentially much more complicated. Don't worry about it until deployment.

The second one is for local usage (Jupyter runs on `localhost`). In that case a simple file selection widget suffices. 

Go with the second one for the presentation.

## For presentation: File selection

Note: The simplest solution would be to just provide a static text box in which the user should paste the rel. or abs. filepath. However, I want a graphical filebrowser. Preferably one that uses the system file browser (Qt via PySide2 over Tkinter and ipywidgets).

## For deployment: File upload
For the deployment as a standalone app, the first one has to be used.
Could do it like this:
- upload file to code base in a tmp folder
- delete file policy:
  - either user-selection or when app is closed (how?)
  - or a cron job that deletes files regularly

# Try out different Jupyter FileChooser solutions

FileChooser synonyms:
- file selector
- file selection widget
- file browser

## NO: With HTML5 + JS only
Source: https://stackoverflow.com/questions/27551399/ipython-notebook-open-file-dialog-retrieve-the-full-path

Problems:
- can't access `fname`
- `file_selector` cannot access absolute filepath due to security

In [5]:
input_form = """
<div style="border:solid navy; padding:20px;">
<input type="file" id="file_selector" name="files[]"/>
<output id="list"></output>
</div>
"""

javascript = """
<script type="text/Javascript">
  function handleFileSelect(evt) {
    var kernel = IPython.notebook.kernel;
    var files = evt.target.files; // FileList object
    console.log('Executing orig')
    console.log(files)
    // files is a FileList of File objects. List some properties.
    var output = [];
    var f = files[0]
    output.push('<li><strong>', escape(f.name), '</strong> (', f.type || 'n/a', ') - ',
                  f.size, ' bytes, last modified: ',
                  f.lastModifiedDate ? f.lastModifiedDate.toLocaleDateString() : 'n/a',
                  '</_Mli>');
    document.getElementById('list').innerHTML = '<ul>' + output.join('') + '</ul>';
    var command = 'fname = "' + f.name + '"'
    console.log(command)
    kernel.execute(command);
  }

  document.getElementById('file_selector').addEventListener('change', handleFileSelect, false);
</script>
"""

def file_selector():
    from IPython.display import HTML, display
    display(HTML(input_form + javascript))


file_selector()


## NO: Custom-built widget from DrDub
Reference:
- https://gist.github.com/DrDub/6efba6e522302e43d055
- http://duboue.net/blog14.html

See also the correction tips there!

Problems:
- no widget to display file path

In [14]:
import os

import ipywidgets as widgets


class FileBrowser(object):
    def __init__(self):
        self.path = os.getcwd()
        self._update_files()
        
    def _update_files(self):
        self.files = list()
        self.dirs = list()
        if(os.path.isdir(self.path)):
            for f in os.listdir(self.path):
                ff = self.path + "/" + f
                if os.path.isdir(ff):
                    self.dirs.append(f)
                else:
                    self.files.append(f)
        
    def widget(self):
        box = widgets.VBox()
        self._update(box)
        return box
    
    def _update(self, box):
        
        def on_click(b):
            if b.description == '..':
                self.path = os.path.split(self.path)[0]
            else:
                self.path = self.path + "/" + b.description
            self._update_files()
            self._update(box)
        
        buttons = []
        if self.files:
            button = widgets.Button(description='..', background_color='#d0d0ff')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.dirs:
            button = widgets.Button(description=f, background_color='#d0d0ff')
            button.on_click(on_click)
            buttons.append(button)
        for f in self.files:
            button = widgets.Button(description=f)
            button.on_click(on_click)
            buttons.append(button)
        box.children = tuple([widgets.HTML("<h2>%s</h2>" % (self.path,))] + buttons)

In [15]:
# example usage:
f = FileBrowser()
f.widget()
#   <interact with widget, select a path>

In [16]:
# in a separate cell:
f.path # returns the selected path

'/home/johannes/Desktop/Studium/Kurse_RWTH/SiScLab/18W/repos/jupyter/appmode'

## YES: Using `Tkinter`
Reference: https://codereview.stackexchange.com/questions/162920/file-selection-button-for-jupyter-notebook

Problems:
- This works fine for me (JupyterLab, Python 3.5)!

In [5]:
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog


class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [8]:
my_button = SelectFilesButton()

In [10]:
display(my_button) # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [52]:
# In a different cell of the same Jupyter Notebook You can access the file list by using the following:
my_button.files

['/home/johannes/credentials.htpasswd']

### Extension: display selected filepaths in textfields

In [51]:
import os
tfbox = widgets.VBox()
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self, tfbox):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "red"
        # Set on click behavior.
        self.on_click(self.select_files)
        self.tfbox = tfbox

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "green"
        
        tfbox.children = [widgets.Text(layout={'width' : 'initial'}) for i in range(len(b.files))]
        for (tfield, filepath) in zip(tfbox.children, b.files):
            tfield.value = os.path.relpath(filepath)
            
my_button = SelectFilesButton(tfbox)
display(my_button)
display(tfbox)

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='red'))

VBox()

## MAYBE: Using Qt
Benefits:
- use system file browser instead of some custom implementation

References:
- https://stackoverflow.com/q/27551399 links to
  - example https://stackoverflow.com/questions/20790926/ipython-notebook-open-select-file-with-gui-qt-dialog
  - issue with qt+matplotlib in same notebook https://github.com/ipython/ipython/issues/5798
    - example notebook reproducing the issue https://nbviewer.jupyter.org/gist/tritemio/430d68d8b0efbbb13462
- https://groups.google.com/forum/#!topic/jupyter/EmsrRq3bs3M

Use PySide2 instead PySide as of 2018-12:
- https://wiki.qt.io/Qt_for_Python
- https://anaconda.org/conda-forge/pyside2

Didn't get any example running. Abandoned.

### NO: using separate py script path
Reference: https://stackoverflow.com/a/46814297

Problems:
- doesn't work for me (JupyterLab, Python 3.5)

#### (needed for import: setup masci-tools path)

In [2]:
# IMPORTANT: we need to import stuff from masci-tools folder.
# Since masci-tools is not installed as a module (yet), the notebook kernel
# needs to be started in the masci-tools folder.
# If that has not happened for some reason, then need to add the masci-tools
# manually to the sys path.
import os
import sys

cwd = os.getcwd()
path_mtools = cwd
dirname_mtools = "masci-tools"
# first try if we can get away without needing an absolute path
if dirname_mtools in path_mtools:
    while os.path.basename(path_mtools) != dirname_mtools:
        path_mtools = os.path.split(path_mtools)[0]
else:
    # okay, try with an absolute path
    path_mtools = "/home/johannes/Desktop/Studium/Kurse_RWTH/SiScLab/18W/repos/masci-tools"
    if not os.path.isdir(path_mtools):
        raise IOError(f"Could not find path to masci-tools. Please specify absolute path.")

# found masci-tools. add to syspath (for imports) and chdir.
if path_mtools not in sys.path:
    # add only once
    sys.path.append(path_mtools)

this does not work: the cell loading does not finish.

In [ ]:
from studenproject18ws.jupyter import example_filechoosers_qt
example_filechoosers_qt.gui_fname()

### MAYBE: using Qt directly
- used PySide2 instead of PySide
- got it to work using [this](https://nbviewer.jupyter.org/gist/tritemio/430d68d8b0efbbb13462)

Problems:
- need to adapt so I can call it via a button

In [10]:
from PySide2 import QtWidgets

def gui_fname(dir=None):
    """Select a file via a dialog and return the file name.
    """
    if dir is None: dir ='./'
    fname = QtWidgets.QFileDialog.getOpenFileName(None, "Select data file...", 
            dir, filter="All files (*);; SM Files (*.sm)")
    return fname[0]

In [11]:
%gui qt

In [12]:
%gui qt
file_name = gui_fname()
%matplotlib widget

In [8]:
file_name

'/home/johannes/Desktop/Studium/Kurse_RWTH/SiScLab/18W/repos/masci-tools/studenproject18ws/data/input/banddos_4x4.hdf'

#### WIP: Reproduce as widget button like the `Tkinter` example above
Tried to combine th Tkinter ipywidgets button with the Qt dialog above.

Doesn't work right now.

In [27]:
import traitlets
from ipywidgets import widgets
from IPython.display import display
from PySide2 import QtWidgets

class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # List of selected fileswill be set to b.value
        if dir is None: dir ='./'
        b.files = QtWidgets.QFileDialog.getOpenFileNames(None, "Select data file...", 
            dir, filter="All files (*);; SM Files (*.sm)")
        b.files[0]

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [28]:
%gui qt

In [29]:
%gui qt

my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

#### Second attempt

In [31]:
%gui qt

In [36]:
from ipywidgets import widgets
from IPython.display import display
from PySide2 import QtWidgets

button = widgets.Button(description="Click Me!")


def on_button_clicked(b):
    print("clicked")
    fname = QtWidgets.QFileDialog.getOpenFileName(None, "Select data file...", 
            dir, filter="All files (*);; SM Files (*.sm)")
    print(fname[0])

button.on_click(on_button_clicked)


In [37]:
display(button)

Button(description='Click Me!', style=ButtonStyle())